#Scripted Primo Search Assessment Tool

Description: < Describe the purpose of the SPSAT>

## Load required libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import glob
import os
import codecs
import json
import time
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus

##Define Some Variables

In [2]:
## URLs ##

# prod_url = <base url>
prod_url_base = 'http://bu-primo.hosted.exlibrisgroup.com:1701/PrimoWebServices/xservice/search/brief'
# stage_url = <base url>
stage_url_base = 'http://bu-primostage.hosted.exlibrisgroup.com:1701/PrimoWebServices/xservice/search/brief'

## query params ##
query_Params1 = '?institution=BOSU&query=any,contains,'
query_Params2 = '&indx=1&bulkSize=10&loc=local,scope,BOSU&loc=adaptor,primo_central_multiple_fe&onCampus=true&json=true'
stage = True
#production = True
#num_results = 10
#scope = 

## file locations ## 

# move to working directory, this is the path where we want files to reside
os.chdir('/Volumes/jwa_drive1/git/spst')

# Input file (contains search strings)
f = '/Volumes/jwa_drive1/git/spst/search_strings.csv'

##Define functions

In [3]:
# build_url 
    # takes base_url and search_string, returns url properly formated for searching
def build_url(base_url,search_string):
    url = base_url+query_Params1 + search_string.replace(' ','+') + query_Params2
    return(url)
    
# parse_response 
    # takes response (as json) from search, returns response_dict (dict of our responses)
def parse_response(response):
    response_dict = {}
    # parse the response to return the data desired elements
    response = json.loads(response.decode('utf8'))
    # need to extract the desired elements and add them to a dict
    response = response['SEGMENTS']
    response = response['JAGROOT']
    response = response['RESULT']
    response_dict['facet'] = response['FACETLIST']
    response_dict['docset'] = response['DOCSET']
    doc = response_dict['docset']['DOC'][0]
    return_dict = {}
    return_dict['TotalHits'] = response['DOCSET']['@TOTALHITS']
    return_dict['FirstHit'] = response['DOCSET']['@FIRSTHIT']
    return_dict['LastHit'] = response['DOCSET']['@LASTHIT']
    return_dict['Rank'] = doc['@RANK']
    return_dict['SearchEngine'] = doc['@SEARCH_ENGINE']
    return_dict['ResultNumber'] = doc['@NO']
    return_dict['Bib'] = doc['PrimoNMBib']['record']

    return(return_dict)

def get_value(response, key):
    # find or navigate response until the key is found
    # then return the value associated with the key
    value = 'hello world'
    
    return(value)
    

In [4]:
# open input file, read each line as a separate string, and remove the new line character
search_strings = open(f).read().splitlines()

In [5]:
# create a DataFrame. Expand the columns to match the desired elements
# to be returned by the function 'parse_response'

# Create a dataframe with
# - column for search string 
# - column for response (returned from parse_response fuction)
columns = ['FirstHit', 'LastHit', 'TotalHits','ResultNumber','Rank','SearchEngine','BibKeys','PrimoNMBib']
df = pd.DataFrame(columns = columns)


# Loop through search_strings, execute search, ... [to be done?]
for search_string in search_strings:
    time.sleep(1)
    
    
    # can we use build_url here? 
    #search_url = stage_url_base+query_Params1 + search_url.replace(' ','+') + query_Params2
    search_url = build_url(prod_url_base,search_string)
    request = Request(search_url)
    # maybe we could do ... 
    # request = Request(build_url(stage_url_base,search_url))
    
    
    try:
        response_body = urlopen(request).read() # reads request into response_body var
        d = parse_response(response_body)
        df = df.append(d, ignore_index=True)
        # second dataframe? 
        #df_response = pd.DataFrame([[search_url, parse_response(response_body)]],columns = ['SearchStr', 'Response'])
        #df = df.append(df_response, ignore_index = True)
    #prints errors 
    except Exception as e:
        print(e)

In [7]:
# verify dataframe
df.head()

,FirstHit,LastHit,TotalHits,ResultNumber,Rank,SearchEngine,BibKeys,PrimoNMBib,Bib
0,1,10,2964097,1,0.07,primo_central_multiple_fe,NaN,NaN,"{'control': {'sourcerecordid': '1645937272', '..."
1,1,10,2621895,1,0.07,primo_central_multiple_fe,NaN,NaN,{'control': {'sourcerecordid': '10.1037/a00356...
2,1,10,2678773,1,0.07000001,primo_central_multiple_fe,NaN,NaN,"{'control': {'sourceformat': 'XML', 'sourcerec..."
3,1,10,2717055,1,0.07,primo_central_multiple_fe,NaN,NaN,"{'control': {'sourceformat': 'XML', 'sourcerec..."
4,1,10,2620589,1,0.0155665865,primo_central_multiple_fe,NaN,NaN,"{'control': {'sourceformat': 'XML', 'sourcerec..."


In [6]:
for index,row in df.iterrows():
    print(row['TotalHits'])
    print(row['FirstHit'])
    print(row['LastHit'])
    print(row['Rank'])
    print(row['SearchEngine'])
    print(row['ResultNumber'])
    print(row['Bib'])
    print()

2964097
1
10
0.07
primo_central_multiple_fe
1
{'control': {'sourcerecordid': '1645937272', 'sourceid': 'proquest', 'sourcesystem': 'Other', 'recordid': 'TN_proquest1645937272'}, 'links': {'linktorsrc': '$$Uhttp://search.proquest.com/docview/1645937272/$$EView_record_in_ProQuest_(subscribers_only)', 'openurl': '$$Topenurl_article', 'openurlfulltext': '$$Topenurlfull_article'}, 'sort': {'author': 'Erath, Lex', 'creationdate': '20141208', 'title': 'Murder', 'lso01': '20141208'}, 'delivery': {'delcategory': 'Remote Search Resource', 'fulltext': 'fulltext'}, 'facets': {'frbrgroupid': '7110366565547982891', 'creationdate': '2014', 'topic': 'Torture', 'creatorcontrib': 'Erath, Lex', 'frbrtype': '6', 'jtitle': 'University Wire', 'rsrctype': 'newspaper_articles', 'language': 'eng', 'prefilter': 'newspaper_articles', 'collection': ['ProQuest Education Journals', 'ProQuest Central', 'ProQuest Social Sciences Premium Collection']}, 'search': {'scope': ['1006080', '1006454', '1006769', '1006907', '